# Parallaxenkorrektur der Startpunkt

Nach der Fallauswahl

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

import matplotlib.pyplot as plt
%matplotlib inline

from standard_config import *
import datetime as dt

import MSGtools as mst
import msg_coord_conv2 as mcc
from analysis_tools import grid_and_interpolation as gi
from analysis_tools import parallax as px

import geooperations as go

from scipy import ndimage as ndi

## 1. Gefilterte HACI-Fälle für den 24.&thinsp;05.&thinsp;2010 laden

In [ ]:
t = dt.date(2010,5,24)

In [ ]:
haci_cases_20100524 = pd.read_csv("{}/2019-09_bell/haci_cases_20100524_filtered.csv".format(proj_path))

In [ ]:
haci_cases_20100524.head()

## 2. Koordinaten umrechnen

Jetzt haben wir Radarfälle im Radarkoordinatensystem. Als nächstes rechnen wir die in das MSG-Koorindatensystem um.

In [ ]:
slon,slat = mst.get_msg_lon_lat('eu','rss',arch_dir="/vols/satellite/home/lenk/data/SEVIRI/")

In [ ]:
lon00,lat00 = mcc.rado_ij2ll(haci_cases_20100524.l00.values,haci_cases_20100524.c00.values)
lon01,lat01 = mcc.rado_ij2ll(haci_cases_20100524.l01.values,haci_cases_20100524.c01.values)
lon0,lat0 = mcc.rado_ij2ll(haci_cases_20100524.l0.values,haci_cases_20100524.c0.values)
lon1,lat1 = mcc.rado_ij2ll(haci_cases_20100524.l1.values,haci_cases_20100524.c1.values)

In [ ]:
haci_cases_20100524 = haci_cases_20100524.assign(lon00 = lon00,
                                                 lat00 = lat00,
                                                 lon01 = lon01,
                                                 lat01 = lat01,
                                                 lon0 = lon0,
                                                 lat0 = lat0,
                                                 lon1 = lon1,
                                                 lat1 = lat1)

In [ ]:
msg_00 = gio.get_index_kdtree(np.array([[lat00],[lon00]]),slat,slon)
msg_01 = gio.get_index_kdtree(np.array([[lat01],[lon01]]),slat,slon)
msg_0 = gio.get_index_kdtree(np.array([[lat0],[lon0]]),slat,slon)
msg_1 = gio.get_index_kdtree(np.array([[lat1],[lon1]]),slat,slon)

In [ ]:
msg_r00 = []
msg_c00 = []
msg_r0 = []
msg_c0 = []
msg_r01 = []
msg_c01 = []
msg_r1 = []
msg_c1 = []

for i,idx in enumerate(msg_00):
    msg_r00.append(idx[0][0])
    msg_c00.append(idx[1][0])
    
    msg_r01.append(msg_01[i][0][0])
    msg_c01.append(msg_01[i][1][0])
    
    msg_r0.append(msg_0[i][0][0])
    msg_c0.append(msg_0[i][1][0])
    
    msg_r1.append(msg_1[i][0][0])
    msg_c1.append(msg_1[i][1][0])

In [ ]:
haci_cases_20100524 = haci_cases_20100524.assign(msg_r00 = msg_r00,
                                                 msg_c00 = msg_c00,
                                                 msg_r01 = msg_r01,
                                                 msg_c01 = msg_c01,
                                                 msg_r0 = msg_r0,
                                                 msg_c0 = msg_c0,
                                                 msg_r1 = msg_r1,
                                                 msg_c1 = msg_c1)

In [ ]:
haci_cases_20100524.to_csv("{}/2019-09_bell/haci_cases_20100524_filtered.csv".format(proj_path),index=False)

In [ ]:
haci_cases_20100524

Als nächstes gehen wir die Fälle durch und machen eine Parallaxenkorrektur der Fallkoordinaten. Dazu brauchen wir zuerst Wolkenhöhenwerte.

In [ ]:
times = haci_cases_20100524.time.values

In [ ]:
ctth_data = {t:[] for t in times}

In [ ]:
for t in times:
    tt = dt.datetime(2010,5,24,int(t.split(":")[0]),int(t.split(":")[1]))
    cth = mst.get_nwcsaf_prod("CTTH_HEIGHT",tt,calibrate=True)
    
    cth_smooth =  ndi.maximum_filter(cth, size = 3)
    cth_smooth =  ndi.gaussian_filter(cth_smooth, 3)
    
    ctth_data[t] = np.rint(cth_smooth).astype("int")

Damit können wir die Koordinaten jetzt korrigieren. Die Korrekturen sollten nicht allzugroß sein. Der maximalwert der NWC&thinsp;SAF-Wolkenhöhe liegt bei 14000&thinsp;m. Das sollten in Deutschland nicht mehr als drei MSG-Standardpixel sein.

In [ ]:
lines = []
cols = []
for i,row in haci_cases_20100524.iterrows():
    l0 = int(np.rint((row.msg_r00 + row.msg_r01) // 2))
    c0 = int(np.rint((row.msg_c00 + row.msg_c01) // 2))
    
    cth_0 = ctth_data[row.time][l0,c0]
    
    lat0 = slat[l0,c0]
    lon0 = slon[l0,c0]
    
    lon_corr, lat_corr = px.parallax_correction(lon0, lat0, cth_0, satlon = 9.5)    
    
    coord = gi.get_index_kdtree(np.array([[lat_corr],[lon_corr]]),slat,slon)
    
    lines.append(coord[0][0])
    cols.append(coord[0][1])

In [ ]:
haci_cases_20100524 = haci_cases_20100524.assign(msg_r_corr = lines,msg_c_corr=cols)

In [ ]:
haci_cases_20100524.to_csv("{}/2019-09_bell/haci_cases_20100524_filtered.csv".format(proj_path),index=False)

## 2. Zu den Koordinaten gehörige Objekte suchen

Mit den Koordianten können wir uns aus der Objektmaske jetzt diejenigen Objekte suchen, die zu dem Niederschlag gehören.

In [ ]:
with xr.open_dataset("/vols/satellite/home/lenk/proj/2019-09_bell/objektmaske_20100524_eu.nc") as f:
    objektmaske = f.objektmaske.data

In [ ]:
def time2int(time_string,time_intervall=5):
    hours = (int(time_string.split(":")[0]) * 60) / time_intervall
    minutes = int(time_string.split(":")[1])
    
    return hours + minutes